# First carry out the classification of LDA

In [ ]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import time
import os
import sys
sys.path.append("/Users/qianyuyang/Desktop/paper/code/getCnki/handle_data")
from BAGGING import BAGGING
import jieba
jieba.load_userdict("/Users/qianyuyang/Desktop/paper/data/important/cover_name.txt")
import multiprocessing as mp
import threading
from toolBox import DATA
from get_different_mindf import GetData
DATA = DATA()
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn import metrics
import warnings
import pickle
from scipy.sparse import csr_matrix
import copy
from bidict import bidict
warnings.filterwarnings("ignore")
di = dict()
rr = pd.DataFrame(columns = ["all news accuracy","bad news precision","bad news recall","good news precision","good news recall","f1_score"])
min_df = 5
sample_size = 5
for min_df in [5]:
    if not os.path.exists("min_df = %s"%min_df):
        os.mkdir("min_df = %s"%min_df)
    gd = GetData(min_df = min_df,cut_method="global")
    want_word = gd.want_word
    all_data = gd.data
    all_data = all_data.drop_duplicates(["sentence"])
    all_data["id"] = all_data.index
    
    """
    Here the overall tf-idf matrix is ​​processed
    """
    
    all_tf = TfidfVectorizer()
    all_tf_parse_matrix = all_tf.fit_transform(all_data.text.tolist())
    word_news_parse_matrix = csr_matrix((np.array([1] * len(all_tf_parse_matrix.data)),all_tf_parse_matrix.indices,all_tf_parse_matrix.indptr),shape = all_tf_parse_matrix.shape)
    all_word_list = all_tf.get_feature_names()
    c = copy.deepcopy(all_data)
    c["map_index"] = list(range(len(c)))
    mp = c["map_index"].to_dict()
    rmp = dict(zip(mp.values(),mp.keys()))
    stop_word_list = DATA.stop_word_list
    cover_name_list = DATA.cover_name_list

di = dict()
min_df = 5
gd = GetData(min_df = min_df,cut_method="global")     
want_word = gd.want_word
all_data = gd.data
all_data = all_data.drop_duplicates(["sentence"])
all_data["id"] = all_data.index
for seed in range(sample_size):
    stat = pickle.load(open("/Users/qianyuyang/Desktop/paper/code/getCnki/handle_data/test21/21.1/min_df = %s/para_dict/para_dict_%s.pkl"%(min_df,seed),"rb"))
    best_key = max(stat,key = stat.get)
    di[seed] = pd.read_csv("/Users/qianyuyang/Desktop/paper/code/getCnki/handle_data/test21/21.1/min_df = %s/%s_%s_wv.csv"%(min_df,seed,best_key))
word = set(di[0][di[0].score < 0].word.tolist())
ss = dict()
for i in range(0,sample_size):
    data = di[i]
    sss = set(data[data.score<0].word.tolist())
    for j in sss:
        if j not in ss.keys():
            ss[j] = 0
        ss[j] += 1
ws = set(sorted(ss,key=ss.__getitem__,reverse=False))
wl = [k for k,v in ss.items() if v >= 1]
predict_data = pd.read_pickle("/Users/qianyuyang/Desktop/paper/code/getCnki/handle_data/test21/21.2/predict_5_3.pkl")
bad_data = predict_data[predict_data.label == -1]
bad_data.text = bad_data.text.apply(lambda x:" ".join([i for i in x.split(" ") if i in set(wl)]))

words_ls = bad_data.text.apply(lambda x: x.split(" ")).tolist()

import jieba
import jieba.posseg as jp
from gensim import corpora, models
bad_data["kind"] = 0


# Generate corpus dictionary
dictionary = corpora.Dictionary(words_ls)
# generate sparse vector set
corpus = [dictionary.doc2bow(words) for words in words_ls]
# LDA model, num_topics sets the number of clusters, that is, the number of final topics
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=6)
import collections
from wordcloud import WordCloud
mytext = dict()
for topic in lda.print_topics(num_words=1000):
    #print(topic)
    if topic[0] not in mytext.keys():
        mytext[topic[0]] = ""
    exl = topic[1].split(" + ")
    for i in exl:
        ie = i.split("*")
        r = float(ie[0])
        word = ie[1].replace('"',"")
        mytext[topic[0]] += int(1000 * r) * ("%s "%word)
    word_counts = collections.Counter(mytext[topic[0]].split(" "))
    wc = WordCloud(font_path="Hiragino Sans GB.ttc",background_color = "white").generate(mytext[topic[0]])
        
        
    wc.generate_from_frequencies(word_counts)
    %pylab inline

    plt.imshow(wc, interpolation='bilinear')
    
    plt.axis("off")
    plt.show()
print('推断:')
for e, values in enumerate(lda.inference(corpus)[0]):
    topic_val = 0
    topic_id = 0
    
    for tid, val in enumerate(values):
        
        if val > topic_val:
            topic_val = val
            topic_id = tid
    bad_data.kind.iloc[e] = topic_id



# Classify

In [ ]:
kind_dict = {
    0:"违规",
    1:"上市状态变更",
    2:"违规",
    3:"行情异动",
    4:"造假与内幕交易",
    5:"法律诉讼与刑事责任"
}
bad_data.kind = bad_data.kind.map(kind_dict)

# Statistical testing unit

In [ ]:

def stat_test(k):
    import pandas as pd
    import pickle
    import warnings
    warnings.filterwarnings("ignore")
    import os


    final_res = pd.DataFrame(columns = ["Y","X","T","start_value","end_value","end_value - start_value","end_value / start_value"])
    res_stat = {
        0:[],
        1:[],
        2:[],
        3:[],
        4:[],
        5:[],
        6:[]
    }
    rs = pd.DataFrame(columns = ["Y","X","T","start_value","end_value","end_value - start_value","end_value / start_value"])
    relations = pd.DataFrame(columns = ["Y","X","T","start_value","end_value","end_value - start_value","end_value / start_value"])
    for t in range(1,5):
        data1 = pickle.load(open("43_0.3_exp.pkl","rb"))
        data2 = pickle.load(open("96_0.9_exp.pkl","rb"))
        data3 = pickle.load(open("77_1.0_exp.pkl","rb"))
        k = pd.merge(data1[["id","label"]],data2[["id","label"]],on = "id",how = "outer")
        k = pd.merge(k,data3[["id","label"]],on = "id",how = "outer")
        #k = k.fillna(0)
        k.rename(columns = {"label":"label_z"},inplace = True)
        k = k.dropna()
        k["stat"] = k["label_x"] + k["label_y"] + k["label_z"]
        k["label"] = k["stat"].apply(lambda x: x//3 if x % 3 == 0 else x)

        all_data = pd.read_pickle("/Users/qianyuyang/Desktop/paper/data/important/sentence_target_cut_already.pkl")
        all_data.dropna(inplace = True)
        data = pd.merge(all_data,k[["id","label"]],left_index = True,right_on="id",how = "left")
        data.dropna(inplace = True)

        news_hash = pd.read_pickle("/Users/qianyuyang/Desktop/paper/data/important/news_hash.pkl")
        date_hash = news_hash["date"].to_dict()
        data.news_id = data.news_id.astype(int)
        data["date"] = data["news_id"].map(date_hash)

        data.date = data.date.astype(int)
        data.date = data.date.astype(str)

        data = data[data.date < str(20211231 - 10000 * t)]

        period_list = sorted(pd.read_pickle("/Users/qianyuyang/Desktop/allStockFactor/HQ/CLOSE.pkl").index)
        import datetime
        def handle_date(x):
            global period_list
            period_signal = ""
            
            for j in range(len(period_list)):
                if period_list[j] > x:
                    period_signal = period_list[j - 1]
                    break
            front_distance = (datetime.datetime(int(x[:4]),int(x[4:6]),int(x[6:])) - datetime.datetime(int(period_signal[:4]),int(period_signal[4:6]),int(period_signal[6:]))).days
            back_distance = (datetime.datetime(int(period_list[j][:4]),int(period_list[j][4:6]),int(period_list[j][6:])) - datetime.datetime(int(x[:4]),int(x[4:6]),int(x[6:]))).days
            return period_signal,front_distance,back_distance,period_list[j + t - 1]


        l = list()
        for i in data.date:

            a = handle_date(i)
            l.append(a)
        
        data["start"] = [i[0] for i in l]
        data["end_1year"] = [i[3] for i in l]
        data["front_distance"] = [i[1] for i in l]
        data["back_distance"] = [i[2] for i in l]
        
        
        
        import copy
        import statsmodels.api as sm



        for f in os.listdir("/Users/qianyuyang/Desktop/allStockFactor"):
            if ".pkl" in f:
                
                cd = copy.deepcopy(data)
                factor_name = f[:-4]
                factor_data = pd.read_pickle("/Users/qianyuyang/Desktop/allStockFactor/%s"%f)
                cd["start_value"] = [factor_data.loc[cd.start.iloc[i],cd.stock_code.iloc[i]] for i in range(len(cd))]
                cd["end_value"] = [factor_data.loc[cd.end_1year.iloc[i],cd.stock_code.iloc[i]] for i in range(len(cd))]
                cd["end-start"] = cd["end_value"] - cd["start_value"]
                cd["end/start"] = cd["end_value"] / cd["start_value"]
                
                cd["bad_signal"] = cd.label.map({1:0,-1:1})
                cd["news_length"] = cd.sentence.apply(lambda x: len(x))
                cd["bad_news_length"] = cd["news_length"] * cd["bad_signal"]
                cd["news_time"] = cd.front_distance / 365
                cd["bad_news_time"] = cd["news_time"] * cd["bad_signal"]
                cd = cd[["label","bad_signal","bad_news_length","bad_news_time","start","stock_code","start_value","end_value","end-start","end/start"]]
                stat = cd.groupby(["stock_code","start"]).agg({
                    "label":"count",
                    "bad_signal":"sum",
                    "bad_news_length":"sum",
                    "bad_news_time":"sum",
                    "start_value":"max",
                    "end_value":"max",
                    "end-start":"max",
                    "end/start":"min"
                })
                
                stat = stat[stat.bad_signal != 0]
                stat["bad_news_average_length"] = stat["bad_news_length"] / stat["bad_signal"]
                stat["bad_news_average_time"] = stat["bad_news_time"] / stat["bad_signal"]
                stat["bad_news_num"] = stat["bad_signal"]
                stat["bad_news_rate"] = stat["bad_signal"] / stat["label"]
                
                
                for x in ["bad_news_num","bad_news_rate","bad_news_average_length","bad_news_average_time"]:
                    add_res = [factor_name,x,t]
                    rs_add = [factor_name,x,t]
                    relation_add = [factor_name,x,t]
                    for y in ["start_value","end_value","end-start","end/start"]:
                        sub_data = stat[[x,y]]
                        sub_data = sub_data[(sub_data != float("inf")) & (sub_data != float("-inf"))]
                        sub_data = sub_data.dropna()
                        Y = sub_data[y]
                        X = sm.add_constant(sub_data[x])
                        model = sm.OLS(Y,X)
                        result = model.fit()
                        add_res.append((result.f_pvalue,result.rsquared))
                        relation_add.append((result.params[1],result.tvalues[1]))
                        
                        p_standard,r_standard = 0,0
                        
                        
                        if result.f_pvalue <= 0.001:
                            p_standard = 3
                        elif result.f_pvalue <= 0.01:
                            p_standard = 2
                        elif result.f_pvalue <= 0.05:
                            p_standard = 1
                            
                        if result.rsquared >= 0.03:
                            r_standard = 3
                        elif result.rsquared >= 0.01:
                            r_standard = 2
                        elif result.rsquared >= 0.005:
                            r_standard = 1
                            
                        rs_add.append((p_standard,r_standard))
                        
                        if p_standard >= 1:
                            res_stat[p_standard + r_standard].append([factor_name,x,t,y])
                    final_res.loc[len(final_res)] = add_res
                    rs.loc[len(rs)] = rs_add
                    relations.loc[len(relations)] = relation_add
                            

In [7]:
all_stat = pd.DataFrame(index = result_stat.keys(),columns = f.Y.unique().tolist())
all_stat.fillna(0,inplace = True)

# Results

In [2]:
import pickle
result_stat = pickle.load(open("result_stat.pkl","rb"))

In [4]:
import pandas as pd
status = '上市状态变更'
final_res,rs,relations = result_stat[status]


f = final_res.copy()
for i in final_res.index:
    for j in ["start_value","end_value","end_value - start_value","end_value / start_value"]:
        stat = final_res.loc[i,j]
        pas = relations.loc[i,j]
        t = 1 if abs(pas[1]) >= 1.96 else 0
        coef = 1 if pas[0] > 0 else -1
        f.loc[i,j] = (stat[0],stat[1],t,coef)
mp = dict()
for i in ["start_value","end_value","end_value - start_value","end_value / start_value"]:
    for j in f.index:
        mp[f.loc[j,i]] = rs.loc[j,i]


def countnozero(x):
    counter = 0
    for i in x:
        if i != 0:
            counter += 1
    return counter

#analysis = pd.DataFrame(columns = ["news_factor","company_factor","factor_kind","stat_score"])
rs_p = pd.DataFrame()
rs_p[["Y","X","T"]] = rs[["Y","X","T"]]
for i in ["start_value","end_value","end_value - start_value","end_value / start_value"]:
    
    rs_p[i] = rs[i].apply(lambda x:x[0])
    
    
rs_p["es"] = rs_p["start_value"].apply(lambda x: 1 if x == 0 else 0) * rs_p["end_value"]
factor_select = rs_p.groupby(["Y","X"])[["es","end_value - start_value","end_value / start_value"]].agg({"es":countnozero,"end_value - start_value":countnozero,"end_value / start_value":countnozero})

selected_f = pd.DataFrame()
selected_rs = pd.DataFrame()
selected_rs_p = pd.DataFrame()

for i in factor_select.index:
    
    
    for num in factor_select.loc[i].tolist():
        #print(num)
        if num >= 2:
            selected_f = pd.concat([selected_f,f[(f["Y"] == i[0]) & (f["X"] == i[1])]])
            selected_rs = pd.concat([selected_rs,rs[(rs["Y"] == i[0]) & (rs["X"] == i[1])]])
            selected_rs_p = pd.concat([selected_rs_p,rs_p[(rs_p["Y"] == i[0]) & (rs_p["X"] == i[1])]])
            continue
        
selected_rs = selected_rs.drop_duplicates()
selected_f = selected_f.drop_duplicates()
selected_rs_p = selected_rs_p.drop_duplicates()

mp = dict()
for i in ["start_value","end_value","end_value - start_value","end_value / start_value"]:
    for j in selected_f.index:
        mp[f.loc[j,i]] = selected_rs.loc[j,i]
#final_res.sort_values(["Y","X","T"]).style.applymap(color, subset=["start_value","end_value","end_value - start_value","end_value / start_value"])




def color(val):
    color = " "
    if mp[val][0] == 3:
        if mp[val][1] == 3:
            color = "orangered"
        elif mp[val][1] == 2:
            color = "tomato"
        elif mp[val][1] == 1:
            color = "salmon"
        elif mp[val][1] == 0:
            color = "mistyrose"
    
    elif mp[val][0] == 2:
        if mp[val][1] == 3:
            color = "dodgerblue"
        elif mp[val][1] == 2:
            color = "deepskyblue"
        elif mp[val][1] == 1:
            color = "skyblue"
        elif mp[val][1] == 0:
            color = "lightskyblue"
    
    elif mp[val][0] == 1:
        if mp[val][1] == 3:
            color = "seagreen"
        elif mp[val][1] == 2:
            color = "mediumseagreen"
        elif mp[val][1] == 1:
            color = "springgreen"
        elif mp[val][1] == 0:
            color = "lightgreen"
    
    return 'background-color: %s' % color


selected_f.sort_values(["Y","X","T"]).style.applymap(color, subset=["start_value","end_value","end_value - start_value","end_value / start_value"])


,Y,X,T,start_value,end_value,end_value - start_value,end_value / start_value
84,CLOSE,bad_news_num,1,"(2.429842994157358e-05, 0.02049442234619747, 1, 1)","(0.0005702894249317693, 0.013038114842374049, 1, 1)","(0.0028034267887198953, 0.010325378914454952, 1, 1)","(0.34636479459643554, 0.0010299521132881573, 0, -1)"
176,CLOSE,bad_news_num,2,"(4.316020583535055e-05, 0.02403668048191765, 1, 1)","(0.002095118761605121, 0.012854828304395327, 1, 1)","(0.01770518425138986, 0.00814855793941116, 1, 1)","(0.29322809640326536, 0.0016056044274394, 0, -1)"
268,CLOSE,bad_news_num,3,"(0.0002197750073145565, 0.024279948571878873, 1, 1)","(5.564270403207852e-05, 0.027272975298582303, 1, 1)","(8.923847328239788e-05, 0.027257459812287776, 1, 1)","(0.8233153549041151, 8.974434640374174e-05, 0, 1)"
360,CLOSE,bad_news_num,4,"(0.0032326591854836126, 0.0198041464293377, 1, 1)","(0.0005073083268056499, 0.026020943947972275, 1, 1)","(0.000785046921277821, 0.025676645104220408, 1, 1)","(0.6863110734356821, 0.0003761872079482975, 0, 1)"
85,CLOSE,bad_news_rate,1,"(0.029786934024439722, 0.005471952435248673, 1, -1)","(0.025574649469706938, 0.005495674677108764, 1, -1)","(0.027231300631247464, 0.005650545683147601, 1, -1)","(0.03410885807320426, 0.005203425036872433, 1, -1)"
177,CLOSE,bad_news_rate,2,"(0.12951792779950558, 0.003337359697375919, 0, -1)","(0.07686191504907885, 0.004269788322299628, 0, -1)","(0.06533825738802528, 0.004928055141690035, 0, -1)","(0.00023849821984933152, 0.019445529934936867, 1, -1)"
269,CLOSE,bad_news_rate,3,"(0.21075909218117403, 0.002815477116429266, 0, -1)","(0.095621249251832, 0.004715818553373663, 0, -1)","(0.10044482575806649, 0.004845941824455302, 0, -1)","(3.767505343957445e-05, 0.030113367746627318, 1, -1)"
361,CLOSE,bad_news_rate,4,"(0.39626717284888047, 0.0016586545303973743, 0, -1)","(0.10661471807558702, 0.005662629791135854, 0, -1)","(0.08211471920058093, 0.006948152494753557, 0, -1)","(0.0003112110698054684, 0.0295469385129421, 1, -1)"
89,CQBL,bad_news_rate,1,"(0.0013755125768963713, 0.011037104483098581, 1, -1)","(0.5329639577851144, 0.0004306408560120767, 0, -1)","(0.04943137635864795, 0.00428754537743814, 1, 1)","(0.002804572149183508, 0.009890035994142288, 1, 1)"
181,CQBL,bad_news_rate,2,"(0.005402642727710518, 0.0104112404032779, 1, -1)","(0.014744532148095017, 0.008196995553477304, 1, -1)","(0.9730014517684664, 1.5964089272735293e-06, 0, -1)","(0.19520520164999533, 0.002335788049643517, 0, 1)"


In [5]:
a = selected_rs_p.groupby(["Y","X"])[["es","end_value - start_value","end_value / start_value"]].agg({"es":countnozero,"end_value - start_value":countnozero,"end_value / start_value":countnozero})
b = selected_rs_p.groupby(["Y","X"])[["es","end_value - start_value","end_value / start_value"]].agg({"es":"sum","end_value - start_value":"sum","end_value / start_value":"sum"})
c = b[a >= 2].fillna(0)
analysis = pd.DataFrame(columns = ["company_factor","news_factor","factor_kind","stat_score"])
for i in c.index:
    for j in ["es","end_value - start_value","end_value / start_value"]:
        if c.loc[i,j] > 0:
            analysis.loc[len(analysis)] = (i[0],i[1],j,c.loc[i,j]) if j!= "es" else (i[0],i[1],"end_value",c.loc[i,j])
analysis.to_excel("%s.xlsx"%status)
analysis

,company_factor,news_factor,factor_kind,stat_score
0,CLOSE,bad_news_num,end_value - start_value,9.0
1,CLOSE,bad_news_rate,end_value / start_value,10.0
2,CQBL,bad_news_rate,end_value / start_value,5.0
3,EPS,bad_news_num,end_value - start_value,8.0
4,QBZCXJHSL,bad_news_average_length,end_value / start_value,2.0
5,ROA,bad_news_num,end_value,4.0
6,ROE,bad_news_num,end_value,4.0
7,TURNOVER,bad_news_num,end_value,5.0
8,TURNOVER,bad_news_rate,end_value - start_value,3.0
9,XSMLL,bad_news_rate,end_value / start_value,2.0


In [10]:
sal = analysis.groupby(["company_factor"]).stat_score.sum()
for i in sal.index:
    all_stat.loc[status,i] = sal.loc[i]
all_stat

,QBZCXJHSL,ZVALUE,EPS,YYZB_ZZC,XSMLL,XSJLL,ZCFZL,JYYWBL,ZZCZZL,YFZKZZL,...,QYCS,ROA,YYZBZZL,XJZZL,ROE,TURNOVER,LDBL,UPDOWN,CLOSE,CQBL
法律诉讼与刑事责任,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
造假与内幕交易,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
违规,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
行情异动,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
上市状态变更,2.0,0,8.0,0,2.0,0,2.0,0,2.0,0,...,0,4.0,0,0,4.0,8.0,0,0,19.0,5.0
